In [32]:
import numpy as np
import pandas as pd
import ast
import sklearn.feature_extraction.text as sk_text
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import pickle

In [33]:
movies = pd.read_csv('tmdb_5000_movies.csv',on_bad_lines='skip')
credits = pd.read_csv('tmdb_5000_credits.csv', on_bad_lines='skip')


In [34]:
movies = movies.merge(credits,on='title')
movies = movies[['title','overview','genres','keywords','cast','crew','movie_id']]
movies.isnull().sum()
movies.duplicated().sum()
movies.dropna(inplace=True)

In [35]:
def func1(obj):
  l = []
  count = 0
  for i in ast.literal_eval(obj):
    if count !=3:
      l.append(i['name'])
      count += 1
    else:
      break
  return l

In [36]:
movies['genres'] = movies['genres'].apply(func1)
movies['keywords'] = movies['keywords'].apply(func1)

In [37]:
movies['cast'] = movies['cast'].apply(func1)

In [38]:
def func2(obj):
  l = []
  count = 0
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      l.append(i['name'])
      break
  return l

In [39]:
movies['crew'] = movies['crew'].apply(func2)
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [40]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies.head()

,title,overview,genres,keywords,cast,crew,movie_id
0,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy]","[cultureclash, future, spacewar]","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],19995
1,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland]","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],285
2,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent]","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],206647
3,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama]","[dccomics, crimefighter, terrorist]","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],49026
4,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion]","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],49529


In [41]:
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']
movies['tags'] = movies['tags'].apply(lambda x:" ".join(x))
movies['tags'] = movies['tags'].apply(lambda x:x.lower())
new_movies = movies[['title','movie_id','tags']]

In [42]:
ps = nltk.PorterStemmer()
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [43]:
new_movies['tags'] = new_movies['tags'].apply(stem)

/var/folders/2r/8ktcqmtd723f9wgczjhxdn080000gn/T/ipykernel_37531/3229886743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(stem)


In [44]:
cv = sk_text.CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(new_movies['tags']).toarray()
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [45]:
similarity = cosine_similarity(vectors)

In [46]:
def recommend(movies):
    idx = new_movies[new_movies['title']==movies].index[0]
    lst = list(enumerate(similarity[idx]))
    lst = sorted(lst,key = lambda x:x[1],reverse=True)
    lst = lst[1:11]
    l = []
    for i in lst:
        l.append(new_movies.iloc[i[0]]['title'])
    return l

In [48]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [47]:
pickle.dump(new_movies,open('movies.pkl','wb'))